**Install**
```fish
python3 -m venv  .venv
source .venv/bin/activate.fish
pip install -r face/CoreML\ Face\ Tracking/requirements.txt
python -m ipykernel install --user --name=.venv
```

http://localhost:8889/tree?token=57711336f39e4c46af81a847eb15b251

In [1]:
import os
import requests

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import  transforms
from torchvision.datasets import LFWPeople
from torchvision.models import resnet18
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import coremltools as ct

import pickle
import wandb



scikit-learn version 1.4.2 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
Torch version 2.3.0+cu121 has not been tested with coremltools. You may run into unexpected errors. Torch 2.2.0 is the most recent version that has been tested.
Failed to load _MLModelProxy: No module named 'coremltools.libcoremlpython'


In [2]:
os.environ["WANDB_API_KEY"] = "4b28b7410bc92bce660b446e56bd56f33dca3e44"


In [3]:

# Define transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to a common size
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pretrained models
])

# Load dataset
lfw_people_train = LFWPeople(root='./data', split='train', transform=transform, download=True)
lfw_people_test = LFWPeople(root='./data', split='test', transform=transform, download=True)

# Data loader
train_loader = DataLoader(lfw_people_train, batch_size=64, shuffle=True)
test_loader = DataLoader(lfw_people_test, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# Load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280

# Anchor generator for the FPN which by default has 5 feature maps
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# Define the model
model = FasterRCNN(backbone,
                   num_classes=2,  # Background and face
                   rpn_anchor_generator=anchor_generator)

model.to(device)

/home/simon2/dev/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/simon2/dev/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
if not os.path.exists('./models'):
    os.mkdir('./models')

def saveml(model, filepath):
    example = torch.rand(1, 3, 128, 128).to(device)
    traced = torch.jit.trace(model, example) 
    traced.save(filepath)


def save(model, filepath):
    # Extract the state dictionary from the model
    model_state_dict = model.state_dict()

    # Open a file in binary-write mode
    with open(filepath, 'wb') as file:
        # Use pickle to dump the model state dictionary into the file
        pickle.dump(model_state_dict, file)

def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    wandb.log({"test_accuracy": accuracy, "test_precision": precision, "test_recall": recall, "test_f1_score": f1})

    return accuracy, precision, recall, f1



In [6]:
def initialize_wandb(config):
    wandb.init(project='face', config=config)

def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    initialize_wandb({
        'learning_rate': 0.001,
        'epochs': num_epochs,
        'batch_size': 64,
        'optimizer': 'Adam'
    })

    model.train()

    for epoch in range(num_epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Converting outputs to probabilities and predictions
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(probs, 1)

            # Calculate metrics per batch
            accuracy = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())
            precision = precision_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro', zero_division=0)
            recall = recall_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro', zero_division=0)
            f1 = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro', zero_division=0)

            # Log metrics per batch
            wandb.log({
                "batch_loss": loss.item(),
                "batch_accuracy": accuracy,
                "batch_precision": precision,
                "batch_recall": recall,
                "batch_f1_score": f1
            })

        print(f'Epoch {epoch + 1}/{num_epochs} completed, Loss: {loss.item()}')
        wandb.log({"epoch": epoch + 1, "epoch_loss": loss.item()})
        saveml(model, './models/model.mlmodel')
        save(model, './models/model.pth')


    print('Finished Training')


In [7]:
train_model(model, train_loader, nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=0.001), num_epochs=5)

wandb: Currently logged in as: pre63. Use `wandb login --relogin` to force relogin


/home/simon2/dev/.venv/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/simon2/dev/.venv/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/5 completed, Loss: 7.754355430603027
Epoch 2/5 completed, Loss: 7.149600505828857
Epoch 3/5 completed, Loss: 7.164018630981445
Epoch 4/5 completed, Loss: 6.79644250869751
Epoch 5/5 completed, Loss: 4.920729637145996
Finished Training


In [8]:
evaluate_model(model, test_loader)

/home/simon2/dev/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/simon2/dev/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.0, 0.0, 0.0, 0.0)

In [9]:
import coremltools
import PIL.Image

def load_image(image_path, resize_to):
    """Load an image from the file system and resize it."""
    image = PIL.Image.open(image_path)
    image = image.resize(resize_to, PIL.Image.ANTIALIAS)
    return image

def prepare_image(image):
    """Prepare the image as required by the CoreML model."""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return image

def predict_image(model_path, image_path):
    """Load the CoreML model, prepare the image, and make a prediction."""
    # Load the CoreML model
    model = coremltools.models.MLModel(model_path)
    
    # Load and prepare the image
    image = load_image(image_path, (128, 128))  # Resize to the input size expected by the model
    image = prepare_image(image)

    # Make a prediction
    prediction = model.predict({'image': image})
    
    # Extract bounding boxes from the prediction
    if 'coordinates' in prediction:
        bounding_boxes = prediction['coordinates']
    else:
        bounding_boxes = None
    
    return bounding_boxes

# download image fromthe web

sample_image_path = "./data/sample.jpg"

def download_image(url):
    response = requests.get(url)
    file = open(sample_image_path, "wb")
    file.write(response.content)
    file.close()


url = 'https://cdn.britannica.com/79/173779-050-2FC54270/Andrew-Grove-Robert-Noyce-Gordon-Moore-1978.jpg'
download_image(url)

# Example usage
bounding_boxes = predict_image('./models/model.mlmodel', sample_image_path)
print("Bounding Boxes:", bounding_boxes)

DecodeError: Error parsing message with type 'CoreML.Specification.Model'